*This notebook is from https://github.com/neubig/anlp-code* by Graham Neubig

We added additional printing of feature weights in the Error Analysis section.

# Training a Sentiment Classifier

This is a notebook for [CMU CS11-711 Advanced NLP](https://cmu-l3.github.io/anlp-fall2025/) that trains a sentiment classifier based on data. Specifically, it uses a bag-of-words to extract features, and the structured perceptron algorithm to train the classifier.

It will take in a text `X` and return a `label` of "1" if the sentiment of the text is positive, "-1" if the sentiment of the text is negative, and "0" if the sentiment of the text is neutral. You can test the accuracy of your classifier on the [Stanford Sentiment Treebank](http://nlp.stanford.edu/sentiment/index.html) by running the notebook all the way to end.

## Setup

Setup code, do imports.

In [2]:
import random
import tqdm

## Feature Extraction

Feature extraction code, how do we get the features we use in training? By default we just use every word.

In [3]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

Also, initialize the feature weights to zero.

In [4]:

feature_weights = {}

## Data Reading

Read in the data from the training and dev (or finally test) sets

In [5]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [6]:
x_train, y_train = read_xy_data('./data/train.txt')
x_dev, y_dev = read_xy_data('./data/dev.txt')

In [7]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


## Inference Code

How we run the classifier.

In [8]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

## Training Code

Learn the weights of the classifier.

In [9]:
NUM_EPOCHS = 5
for epoch in range(1, NUM_EPOCHS+1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:    
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            for feature in features:
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]

Epoch 5: 100%|██████████| 8544/8544 [00:00<00:00, 294862.66it/s]


## Evaluation Code

How we evaluate the classifier:

In [10]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [11]:
label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{1: 444, 0: 229, -1: 428}


In [12]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.7437968164794008
Dev/test accuracy: 0.5776566757493188


## Error Analysis

An important part of improving any system is figuring out where it goes wrong. The following two functions allow you to randomly observe some mistaken examples, which may help you improve the classifier. Feel free to write more sophisticated methods for error analysis as well.

In [13]:
def get_feature_contributions(features):
    output = {}
    for feat_name, feat_value in features.items():
        output[feat_name] = feat_value * feature_weights.get(feat_name, 0)
    return output

def find_errors(x_data, y_data):
    error_ids = []
    y_preds = []
    id2contributions = {}
    for i, (x, y) in enumerate(zip(x_data, y_data)):
        features = extract_features(x)
        y_preds.append(run_classifier(features))
        if y != y_preds[-1]:
            error_ids.append(i)
            id2contributions[i] = get_feature_contributions(features)
    for _ in range(5):
        my_id = random.choice(error_ids)
        x, y, y_pred = x_data[my_id], y_data[my_id], y_preds[my_id]

        print(f'{x}\ntrue label: {y}\npredicted label: {y_pred}')
        contributions = sorted(id2contributions[my_id].items(), key=lambda x: -x[1])
        for feat_name, contribution in contributions:
            print(f'Feature: {feat_name} ({contribution})')
        
        print()

In [14]:
find_errors(x_dev, y_dev)

Prurient playthings aside , there 's little to love about this English trifle .
true label: -1
predicted label: 0
Feature: love (6.0)
Feature: English (6.0)
Feature: there (4.0)
Feature: aside (1.0)
Feature: Prurient (0.0)
Feature: playthings (0.0)
Feature: this (0.0)
Feature: 's (-1.0)
Feature: to (-1.0)
Feature: . (-1.0)
Feature: , (-2.0)
Feature: trifle (-3.0)
Feature: about (-4.0)
Feature: little (-5.0)

Sam Mendes has become valedictorian at the School for Soft Landings and Easy Ways Out .
true label: -1
predicted label: 1
Feature: Out (7.0)
Feature: Mendes (4.0)
Feature: for (4.0)
Feature: and (4.0)
Feature: has (3.0)
Feature: Sam (1.0)
Feature: valedictorian (0.0)
Feature: the (0.0)
Feature: School (0.0)
Feature: Soft (0.0)
Feature: Landings (0.0)
Feature: Easy (0.0)
Feature: Ways (0.0)
Feature: . (-1.0)
Feature: at (-3.0)
Feature: become (-4.0)

Not since Tom Cruise in Risky Business has an actor made such a strong impression in his underwear .
true label: 1
predicted label: -1

## Visualize feature weights

We can inspect the weights that were learned for various features. Below we show the largest, smallest, and randomly selected feature weights. Inspecting them may give insight into the learned classifier.

In [15]:
import random

k = 25
topk_features = sorted(feature_weights.items(), key=lambda x: -x[1])[:k]
bottomk_features = sorted(feature_weights.items(), key=lambda x: x[1])[:k]
randomk_features = random.sample(list(feature_weights.items()), k)

print("Top-k")
for feature in topk_features:
    print(feature)

print("\nBottom-k")
for feature in bottomk_features:
    print(feature)

print("\nRandom k")
for feature in randomk_features:
    print(feature)

Top-k
('refreshing', 13.0)
('powerful', 13.0)
('remarkable', 12.0)
('solid', 12.0)
('rare', 12.0)
('appealing', 12.0)
('fun', 11.0)
('charming', 11.0)
('ability', 10.0)
('eyes', 10.0)
('works', 10.0)
('manages', 10.0)
('heart', 10.0)
('wonderfully', 10.0)
('always', 10.0)
('wild', 10.0)
('assured', 10.0)
('summer', 10.0)
('enjoyable', 9.0)
('beautifully', 9.0)
('sharp', 9.0)
('amused', 9.0)
('human', 9.0)
('sweet', 9.0)
('brilliant', 9.0)

Bottom-k
('stupid', -17.0)
('worst', -16.0)
('depressing', -13.0)
('pretentious', -13.0)
('devoid', -13.0)
('flat', -12.0)
('contrived', -12.0)
('TV', -12.0)
('none', -12.0)
('tired', -11.0)
('failure', -11.0)
('dull', -10.0)
('scene', -10.0)
('instead', -10.0)
('cold', -10.0)
('mess', -10.0)
('loses', -10.0)
('uneven', -10.0)
('Lawrence', -10.0)
('harder', -10.0)
('left', -9.0)
('rises', -9.0)
('suffers', -9.0)
('too', -9.0)
('advice', -9.0)

Random k
('sends', 5.0)
('kills', -1.0)
('beer', -2.0)
('coherence', -3.0)
('Native', 1.0)
('tattoo', 1.0)
(